# bert-base-uncased

In [10]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install transformers datasets
from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch

# 加載預訓練的模型和分詞器
model_name = 'bert-base-uncased' # Changed to a known working model
# Alternatively, try using AutoTokenizer if the model has a custom tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2) # Added num_labels for binary classification

# 設定為評估模式
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
import pandas as pd
from datasets import Dataset

# 假設您已經將 Fake.csv 和 True.csv 上傳到 Colab
fake_news = pd.read_csv('/content/Fake.csv')
true_news = pd.read_csv('/content/True.csv')

# 標註類別：0 代表假新聞，1 代表真新聞
fake_news['label'] = 0
true_news['label'] = 1

# 合併兩個資料集並打亂
data = pd.concat([fake_news[['text', 'label']], true_news[['text', 'label']]])
data = data.sample(frac=1).reset_index(drop=True)

# 轉換為Dataset格式
dataset = Dataset.from_pandas(data)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# 分詞並處理數據
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/44898 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

def create_dataloader(tokenized_datasets, batch_size=16):
    input_ids = torch.tensor(tokenized_datasets['input_ids'])
    attention_mask = torch.tensor(tokenized_datasets['attention_mask'])
    labels = torch.tensor(tokenized_datasets['label'])

    dataset = TensorDataset(input_ids, attention_mask, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

# 創建dataloader
dataloader = create_dataloader(tokenized_datasets)

def predict_and_evaluate_with_dataloader(model, dataloader, device):
    predictions = []
    labels = []

    for batch in dataloader:
        input_ids, attention_mask, label = [x.to(device) for x in batch]

        # 預測
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

        predictions.extend(preds.cpu().numpy())
        labels.extend(label.cpu().numpy())

    return predictions, labels

# 使用新的方法進行預測
predictions, labels = predict_and_evaluate_with_dataloader(model, dataloader, device)

# 計算評估指標
accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions)
recall = recall_score(labels, predictions)
f1 = f1_score(labels, predictions)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.5408
Precision: 0.5392
Recall: 0.2565
F1 Score: 0.3477


In [ ]:
# 檢查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 將模型移動到GPU
model.to(device)

Using device: cuda


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

# distilbert-base-uncased-finetuned-sst-2-english

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 加載數據
fake_df = pd.read_csv('/content/Fake.csv')
true_df = pd.read_csv('/content/True.csv')

# 添加標籤
fake_df['label'] = 0
true_df['label'] = 1

# 合併數據
data = pd.concat([fake_df, true_df], ignore_index=True)

# 切分訓練和測試數據
X = data['text']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [1]:
# 安裝依賴
!pip install transformers datasets sklearn

# 加載依賴庫
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 加載數據
fake_df = pd.read_csv('/content/Fake.csv')
true_df = pd.read_csv('/content/True.csv')

# 添加標籤
fake_df['label'] = 0
true_df['label'] = 1

# 合併數據
data = pd.concat([fake_df, true_df], ignore_index=True)

# 切分訓練和測試數據
X = data['text']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 使用較小的模型
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Tokenize 測試數據
def tokenize_texts(texts, tokenizer, max_length=128):
    return tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

# 使用一小部分測試數據（如果數據集太大，這步很重要）
X_test_small = X_test[:100]  # 只取前100條數據
y_test_small = y_test[:100]

# Tokenize 測試集
test_encodings = tokenize_texts(X_test_small, tokenizer)

# 批處理推理
def predict_batch(model, encodings, batch_size=16):
    model.eval()  # 設置模型為推理模式
    predictions = []
    with torch.no_grad():
        for i in range(0, len(encodings['input_ids']), batch_size):
            input_batch = {key: value[i:i + batch_size] for key, value in encodings.items()}
            output = model(**input_batch)
            batch_predictions = torch.argmax(output.logits, axis=1).cpu().numpy()
            predictions.extend(batch_predictions)
    return predictions

# 使用批處理進行推理
predictions = predict_batch(model, test_encodings)

# 計算評估指標
accuracy = accuracy_score(y_test_small, predictions)
precision = precision_score(y_test_small, predictions)
recall = recall_score(y_test_small, predictions)
f1 = f1_score(y_test_small, predictions)

# 輸出結果
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Accuracy: 0.4600
Precision: 0.4722
Recall: 0.3269
F1 Score: 0.3864


# new_perfect


In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 加載部分數據
fake_data = pd.read_csv("/content/Fake.csv").head(1000)  # 只取前 1000 行
true_data = pd.read_csv("/content/True.csv").head(1000)  # 只取前 1000 行

# 添加標籤
fake_data['label'] = 0  # 假新聞標籤為 0
true_data['label'] = 1  # 真新聞標籤為 1

# 合併數據
all_data = pd.concat([fake_data, true_data], ignore_index=True)

# 定義自定義數據集類
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

# 加載模型和分詞器
tokenizer = AutoTokenizer.from_pretrained("hamzab/roberta-fake-news-classification")
model = AutoModelForSequenceClassification.from_pretrained("hamzab/roberta-fake-news-classification")

# 構建數據集和數據加載器
dataset = NewsDataset(
    texts=all_data['text'].tolist(),
    labels=all_data['label'].tolist(),
    tokenizer=tokenizer
)

dataloader = DataLoader(dataset, batch_size=4, shuffle=False)  # 減小批量大小

# 評估模型
def evaluate(model, dataloader):
    model.eval()
    model.to("cuda" if torch.cuda.is_available() else "cpu")

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to("cuda" if torch.cuda.is_available() else "cpu")
            attention_mask = batch['attention_mask'].to("cuda" if torch.cuda.is_available() else "cpu")
            labels = batch['label'].to("cuda" if torch.cuda.is_available() else "cpu")

            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    return accuracy, precision, recall, f1

# 計算結果
accuracy, precision, recall, f1 = evaluate(model, dataloader)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Accuracy: 0.9350
Precision: 0.8998
Recall: 0.9790
F1 Score: 0.9377
